In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K


In [2]:
import numpy as np

x = np.zeros((1060, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\TrainingA_1-1060.npy", 'rb') as f1:
        x = np.load(f1)
        

In [3]:
y = np.zeros((1060))
for i in range (1060):
    if (i%2==0):
        y[i] = 0
    else:
        y[i] = 1


In [4]:
x = np.reshape(x, (1060, 340, 444, 6))
y = y.astype(int)


In [5]:
def denseNet(inputShape, nClasses, filters = 32):
    
    def BnRlCv (x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def denseBlock(x, repetition):
        for _ in range(repetition):
            y = BnRlCv(x, 4*filters)
            y = BnRlCv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition(x):
        x = BnRlCv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (inputShape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        d = denseBlock(x, repetition)
        x = transition(d)
    x = GlobalAveragePooling2D()(d)
    x = Dropout(0.3)(x)
    output = Dense(nClasses, activation = 'sigmoid')(x)
    
    model = Model(input, output)
    return model

inputShape = 340, 444, 6
nClasses = 1
model = denseNet(inputShape, nClasses)


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 340, 444, 6) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 170, 222, 64) 18880       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 85, 111, 64)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 85, 111, 64)  256         max_pooling2d[0][0]              
______________________________________________________________________________________________

In [7]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])


In [8]:
history = model.fit(x, y, batch_size = 5, epochs = 18)


Epoch 1/18
212/212 [==============================] - 1086s 5s/step - loss: 0.8081 - accuracy: 0.5509
Epoch 2/18
212/212 [==============================] - 1071s 5s/step - loss: 0.7433 - accuracy: 0.5585
Epoch 3/18
212/212 [==============================] - 1071s 5s/step - loss: 0.7217 - accuracy: 0.5764
Epoch 4/18
212/212 [==============================] - 1073s 5s/step - loss: 0.6727 - accuracy: 0.6415
Epoch 5/18
212/212 [==============================] - 1068s 5s/step - loss: 0.6212 - accuracy: 0.6858
Epoch 6/18
212/212 [==============================] - 1123s 5s/step - loss: 0.5547 - accuracy: 0.7170
Epoch 7/18
212/212 [==============================] - 1148s 5s/step - loss: 0.5190 - accuracy: 0.7585
Epoch 8/18
212/212 [==============================] - 1144s 5s/step - loss: 0.5051 - accuracy: 0.7472
Epoch 9/18
212/212 [==============================] - 1124s 5s/step - loss: 0.4859 - accuracy: 0.7585
Epoch 10/18
212/212 [==============================] - 1125s 5s/step - loss: 0.455

In [9]:
model.save_weights("DenseNetTrainingA.hdf5")